In [1]:
import json
import pickle
import pymysql
import numpy as np
import re

from collections import defaultdict
from lxml import etree
from pymorphy2 import MorphAnalyzer
from time import time

<h2>Constants</h2>

In [2]:
MAX_DISTANCE = 2

RE_E = re.compile('ё', re.U)
RE_S = re.compile('\s+', re.U)

REDUNDANT_TAGS = ['Geox', 'Orgn', 'Trad', 'Qual', 'perf', 'impf', 'pres', 'past', 'futr', 'incl', 'excl', 'Infr',
                  'Slng', 'Arch', 'Litr', 'Inmx', 'Vpre', 'LATN', 'NUMB', 'SYMB', 'UNKN']

KEYBOARD = {'a': 'qwedcxz`', 'b': 'vfghn', 'c': 'xsdfv', 'd': 'xswerfvc', 'e': 'sw234rfd', 'f': 'cdertgbv',
            'g': 'vfrtyhnb', 'h': 'bgtyujmn', 'i': 'ju789olk', 'j': 'nhyuik,m', 'k': 'mjuiol.,', 'l': ',kiop;/.',
            'm': 'nhjk,', 'n': 'bghjm', 'o': 'ki890p;l', 'p': 'lo90-[\';', 'q': '12wsa', 'r': 'de345tgf',
            's': 'zaqwedcx', 't': 'fr456yhg', 'u': 'hy678ikj', 'v': 'cdfgb', 'w': 'aq123eds', 'x': 'zasdc',
            'y': 'gt567ujh', 'z': '`asx', 'а': 'свукепим', 'б': 'ьолдю', 'в': 'чыцукамс', 'г': 'рн678шло',
            'д': 'блшщзж/ю', 'е': 'ак456нрп', 'ж': 'юдщзхэ/', 'з': 'дщ90-хэж', 'и': 'мапрт', 'й': '12цыф',
            'к': 'ву345епа', 'л': 'ьогшщдюб', 'м': 'свапи', 'н': 'пе567гор', 'о': 'трнгшлбь', 'п': 'макенрти',
            'р': 'ипенгоьт', 'с': 'чывам', 'т': 'ипроь', 'у': 'ыц234кав', 'ф': 'йцычя', 'х': 'жз0-=ъеэ',
            'ц': 'й123увыф', 'ч': 'яфывс', 'ш': 'ог789щдл', 'щ': 'лш890зжд', 'ъ': 'эх-=е', 'ы': 'яфйцувсч',
            'ь': 'тролб', 'э': '/жзхъе', 'ю': 'блдж/', 'я': 'фыч'}

<h2>MySQL connector and opencorpora loader</h2>

In [3]:
def get_conn():
    return pymysql.connect(
        host='127.0.0.1',
        unix_socket='/tmp/mysql.sock',
        user='root',
        passwd=None,
        db='spellcheck',
        charset='utf8'
    )


def load_corpora(db=False):
    # lemma = {
    #     id: {
    #         'text': '',
    #         'gram': [
    #             '',
    #             ...
    #         ],
    #         'par': [
    #             {
    #                 'text': '',
    #                 'gram': [
    #                     '',
    #                     ...
    #                 ]
    #             },
    #             ...
    #         ]
    #     }
    # }
    lemma = defaultdict(dict)

    xml_iter = etree.iterparse('dict.opcorpora.xml', events=('start', 'end'))

    conn = get_conn()
    c = conn.cursor()
    if db:
        c.execute("""
            DROP TABLE IF EXISTS `spellcheck`.`word_form`;
            DROP TABLE IF EXISTS `spellcheck`.`lemma`;
            CREATE TABLE `spellcheck`.`lemma` (
              `id`   INT(11)     NOT NULL,
              `gram` VARCHAR(50) NOT NULL,
              PRIMARY KEY (`id`)
            )
              ENGINE InnoDB
              CHARACTER SET utf8;
            CREATE TABLE `spellcheck`.`word_form` (
              `id`       INT(11)      NOT NULL AUTO_INCREMENT,
              `lemma_id` INT(11)      NOT NULL,
              `text`     VARCHAR(37)  NOT NULL,
              `length`   SMALLINT     NOT NULL DEFAULT 0,
              `gram`     VARCHAR(50)  NOT NULL,
              PRIMARY KEY (`id`),
              INDEX `lemma_id_idx` (`lemma_id` ASC),
              CONSTRAINT `lemma_id`
              FOREIGN KEY (`lemma_id`)
              REFERENCES `spellcheck`.`lemma` (`id`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION
            )
              ENGINE InnoDB
              CHARACTER SET utf8;
        """)

    while True:
        act, it = xml_iter.__next__()

        if act == 'start' and it.tag == 'lemma':
            _id = int(it.attrib['id'])
            
            if not _id % 50000:  # for debug usages (total approx 400k)
                print(_id)
            
            # retrieve lemma and its paradigm data
            act, it = xml_iter.__next__()  # <l t="">

            lemma[_id]['text'] = RE_E.sub('е', it.attrib['t'])
            lemma[_id]['gram'] = []
            lemma[_id]['par'] = []

            # retrieve lemma grams
            while True:
                act, it = xml_iter.__next__()  # <g v="">
                if act == 'start' and it.tag == 'g':
                    lemma[_id]['gram'].append(it.attrib['v'])
                    continue
                if act == 'end' and it.tag == 'l':
                    break

            # retrieve lemma paradigm
            while True:
                act, it = xml_iter.__next__()  # <f t="">
                if act == 'start' and it.tag == 'f':
                    # retrieve word form grams
                    wf = {'text': RE_E.sub('е', it.attrib['t']), 'gram': []}
                    while True:
                        act, it = xml_iter.__next__()  # <g v="">
                        if act == 'start' and it.tag == 'g':
                            wf['gram'].append(it.attrib['v'])
                            continue
                        if act == 'end' and it.tag == 'f':
                            break
                    lemma[_id]['par'].append(wf)
                if act == 'end' and it.tag == 'lemma':
                    break

            if db:
                c.execute("""INSERT INTO lemma VALUES (%s, "%s")""" % (_id, ','.join(lemma[_id]['gram'])))
                c.execute("""INSERT INTO word_form (lemma_id, text, length, gram) VALUES """ + ','.join(["""(%s, "%s", %s, "%s")""" % (_id, wf['text'], len(wf['text']), ','.join(wf['gram'])) for wf in lemma[_id]['par']]))

        if act == 'end' and it.tag == 'lemmata':
            break

    if db:
        c.execute("""
            ALTER TABLE `spellcheck`.`word_form`
                ADD INDEX `text` (`text` ASC);
        """)
        conn.commit()
        conn.close()

    return lemma

<h2>PrefixTree class and fuzzy_match function</h2> 

In [4]:
class PrefixTree(object):
    def __init__(self, char='', parent=None):
        self.char = char
        self.parent = parent
        self.children = {}
        self.is_word = False

    def trace(self):
        return (self.parent.trace() if self.parent is not None else '') + self.char

    def _to_list(self):
        if self.is_word:
            yield self.trace()
        for pt in self.children.values():
            for s in pt._to_list():
                yield s

    def __iter__(self):
        return self._to_list()

    def __contains__(self, value):
        if not value:
            return True

        if value[0] not in self.children:
            return False

        return value[1:] in self.children[value[0]]

    def __len__(self):
        return len(self.parent) + 1 if self.parent is not None else 0

    def insert(self, value):
        if not value:
            self.is_word = True
            return

        c = value[0]
        if c not in self.children:
            self.children[c] = PrefixTree(c, self)

        self.children[c].insert(value[1:])


def load_ptree(from_file=True):
    """
    Creates PrefixTree from corpora stored in DB or loads it from pickle serialization file
    """
    if from_file:
        with open('pt.pkl', mode='rb') as pt_pkl:
            pt = pickle.load(pt_pkl)
        return pt

    pt = PrefixTree()
    conn = get_conn()
    c = conn.cursor()
    c.execute("SELECT text FROM word_form")

    inserted = 0  # for debug usages (total approx 5kk)
    for row in c:
        pt.insert(row[0])
        inserted += 1
        if not inserted % 1000000:
            print(inserted)

    with open('pt.pkl', mode='wb') as pt_pkl:
        pickle.dump(pt, pt_pkl)

    return pt

def update_visited(ptree, visited):
    """
    Removes one-word branch starting from leaf, going up to root node, ending in first branching node
    """
    visited[ptree][-1] = 0
    t = ptree.parent

    while t is not None:
        if len(t.children) != 1:
            return
        visited[t][-1] = 0
        t = t.parent


def is_visited(i, ptree, k, visited):
    """
    Checks whether current node was visited within less operations (insert/delete/substitution/transposition)
    """
    d = visited[ptree]
    if -1 in d:  # -1 stands for "node processed completely"
        return True

    m = d.get(i, -1)  # get last distance value for string idx i
    if k > m:
        # proceed further if we came in this node for less operations (current k > last visit k)
        d[i] = k
        visited[ptree] = d
        return False

    return True


def fuzzy_match(s, ptree, k, i=0, visited=None, n=0):
    """
    Computes all strings contained in ptree with a distance <= k
    """
    res = set()

    # handles root node of a ptree
    if ptree.parent is None and ptree.children:
        n = len(s)
        s += '\0' * (k + 1)  # in order to leave an opportunity to insert chars into s
        visited = defaultdict(dict)
        for child in ptree.children.values():
            # main loop, process each starting char in a prefix tree
            res.update(fuzzy_match(s, child, k, i, visited, n))
        return res
    
    # already tried
    if is_visited(i, ptree, k, visited):
        return []

    # can't match
    if k == -1 or (k == 0 and s[i] != ptree.char):
        return []

    if ptree.is_word and (n - i <= k or (n - (i + 1) <= k and ptree.char == s[i])):
        res.add(ptree.trace())
        if not ptree.children:
            update_visited(ptree, visited)
            return res

    if ptree.char != s[i]:
        res.update(fuzzy_match(s, ptree, k - 1, i + 1, visited, n))  # insert s char

    for child in ptree.children.values():
        if n >= i + 2 and s[i + 1] == ptree.char and s[i] == child.char:  # transposition
            if child.is_word and k == 1 and n == i + 2:
                # following transition to grandchild omits the case (in current architecture)
                # when child node forms a valid trace, check it (in upper if) and append manually
                res.add(child.trace())
                if not child.children:
                    update_visited(child, visited)

            for grandchild in child.children.values():
                res.update(fuzzy_match(s, grandchild, k - 1, i + 2, visited, n))

        if ptree.char == s[i]:
            res.update(fuzzy_match(s, child, k, i + 1, visited, n))  # chars are matched, k remains the same
        else:
            res.update(fuzzy_match(s, child, k - 1, i + 1, visited, n))  # substitution

        res.update(fuzzy_match(s, child, k - 1, i, visited, n))  # delete candidate char

    return res

<h2>Weightened Damerau-Levenshtein distance</h2>

In [5]:
def weighted_dam_lev(a, b):
    d = {}
    for i in range(-1, len(a)):
        d[(i, -1)] = i + 1
    for j in range(-1, len(b)):
        d[(-1, j)] = j + 1

    for i in range(len(a)):
        for j in range(len(b)):
            subst_cost = trans_cost = 0
            if a[i] != b[j]:
                subst_cost = .8 if b[j] in KEYBOARD[a[i]] or a[i] in KEYBOARD[b[j]] else 1
                trans_cost = .9

            d[(i, j)] = min([
                d[(i - 1, j)] + 1,  # deletion
                d[(i, j - 1)] + 1,  # insertion
                d[(i - 1, j - 1)] + subst_cost  # substitution
            ])

            if i and j and a[i] == b[j - 1] and a[i - 1] == b[j]:
                d[(i, j)] = min([
                    d[(i, j)],
                    d[(i - 2, j - 2)] + trans_cost
                ])

    return d[(len(a) - 1, len(b) - 1)]

<h2>Filtering candidates according to context (within tags N-grams counts) and Damerau-Levenshtein distance</h2>

In [6]:
def get_word_tags(word, morph):
    if not word:
        return ['PNCT']

    res = {}
    for var in morph.parse(word):
        res[','.join(sorted([t for t in RE_S.sub(',', str(var.tag)).split(',') if t not in REDUNDANT_TAGS]))] = var.score

    return res


def get_ngram_relevance_score(left_t, c_t, right_t, c_t_pool, bi_cnt, tri_cnt):
    l_total = r_total = s_total = 0

    for t in set(c_t_pool):
        l_total += bi_cnt.get('%s+%s' % (left_t, t), 0)
        r_total += bi_cnt.get('%s+%s' % (t, right_t), 0)
        s_total += tri_cnt.get('%s+%s+%s' % (left_t, t, right_t), 0)

    l = 0 if not l_total else float(bi_cnt.get('%s+%s' % (left_t, c_t), 0)) / l_total
    r = 0 if not r_total else float(bi_cnt.get('%s+%s' % (c_t, right_t), 0)) / r_total
    s = 0 if not s_total else float(tri_cnt.get('%s+%s+%s' % (left_t, c_t, right_t), 0)) / s_total

    return .25 * l + .25 * r + .5 * s


def filter_candidates(left, candidates, right, morph, bi_cnt, tri_cnt):
    """
    1. Loop through each combination of left-right tags.
       On each iteration we are trying to find the most relevant tag among each candidate tag.
            2. Relevance is measured as weighted sum of bi/trigram probabilities counted by maximum likelihood
               only among candidates tags.
            3. Relevance is multiplied on a probability of the candidate to belong to concrete tag.
            4. Relevance is multiplied on the probabilities of left/right word tags
            5. Compute log of relevance score and multiply it on (candidate_dam_lev_distance + 1) of candidate.
    6. Repeat 2-5 for each combination of left-right tags.
    7. Find maximum of values computed in 5 and argmax (which is the desired candidate).

    :param left:        left word or None in case of sentence beginning
    :param candidates:  dictionary of candidates {'candidate': candidate_dam_lev_distance}
    :param right:       right word or None in case of sentence ending
    :param morph:       instance of MorphAnalyzer
    :param bi_cnt:      dictionary of tags bigrams counts
    :param tri_cnt:     dictionary of tags trigrams counts
    :return:            most possible candidate
    """
    all_freq = {}

    tags_l = get_word_tags(left, morph)
    tags_r = get_word_tags(right, morph)
    tags_c = {}
    c_t_pool = set()  # pool of distinct candidates tags

    for c in candidates:
        tags_c[c] = get_word_tags(c, morph)
        for t, score in tags_c[c].items():
            c_t_pool.add(t)

    for lt in tags_l:
        for rt in tags_r:
            for c in candidates:
                for ct in tags_c[c]:
                    rel_score = np.log(get_ngram_relevance_score(lt, ct, rt, c_t_pool, bi_cnt, tri_cnt))
                    rel_score += np.log(tags_c[c][ct])
                    rel_score += np.log(tags_l[lt]) + np.log(tags_r[rt])
                    rel_score *= candidates[c]

                    all_freq['%s|%s:%s|%s' % (lt, c, ct, rt)] = rel_score

    print('\n'.join([str(x) for x in sorted(all_freq.items(), key=lambda x: x[1], reverse=True)]))  # for debug usages

    # Each candidate tag for each candidate with relevance score
    # for each left/right tag combination is stored in all_freq for now. Retrieve the best sequence
    most_relevant_seq = max(all_freq, key=lambda k: all_freq[k])

    # retrieve candidate from best sequence
    return most_relevant_seq.split('|')[1].split(':')

<h2>PrefixTree loading from pickle</h2>

In [7]:
st = time()
pt = load_ptree()
print("Loading time: %ss" % (time() - st))

Loading time: 36.91286087036133s


<h2>Counts of OpenCorpora tags N-grams loading from json</h2>

In [8]:
st = time()
tags_bi = json.load(open('bigram.opcorpora.json', encoding='utf-8'))
tags_tri = json.load(open('trigram.opcorpora.json', encoding='utf-8'))
print("Loading time: %ss" % (time() - st))

Loading time: 0.0836181640625s


<h2>Some testing</h2>

In [9]:
st = time()
s = 'прикраснее'
res = s in pt
print("Execution time: %ss\nExists: %s" % (time() - st, res))

st = time()
res = fuzzy_match(s, pt, 2)
print("\nExecution time: %ss\nCandidates count: %s" % (time() - st, len(res)))

ma = MorphAnalyzer()
st = time()
l = 'повести'
r = 'на'
res = filter_candidates(l, res, r, ma, tags_bi, tags_tri)
print("\nExecution time: %ss\n'%s %s %s' ===> '%s %s %s'" % (time() - st, l, s, r, l, res, r))

Execution time: 0.00010633468627929688s
Exists: False

Execution time: 0.17548584938049316s
Candidates count: 11


TypeError: 'set' object is not subscriptable